In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import time
from copy import deepcopy

# Generate Artifical Data

In [2]:
def generate_data2():
    r=5
    n=1000
    dataset=torch.zeros(n,n)
    #np.random.seed(0)
    score=torch.tensor(np.random.randint(1, 6, 10000))
    score_2norm=[]
    for i in score:
        dataset[np.random.randint(0,n)][np.random.randint(0,n)]=i
        score_2norm.append(pow(i,2))
    #torch.manual_seed(2020)
    user2=torch.rand(n,r)
    #torch.manual_seed(2020)
    item2=torch.rand(n,r)
    return dataset,user2,item2,sum(score_2norm)

# Collect and deal with Netflix Dataset

In [3]:
def get_Netflix():
    from collections import defaultdict
    f = open("combined_data_1.txt","r")
    lines = f.readlines()
    lines=lines[:100000]
    movie=[]
    user=defaultdict(list)
    for line in lines:
        item = line.split(',')
        i=0
        if len(item)==1:
            item=item[0].split(":")
            movie.append(int(item[0]))
            movie_name=item[0]
            continue
        user[int(item[0])].append(int(item[1]))
        user[int(item[0])].append(movie_name)
    user_index=list(enumerate(user))
    location_list=[]
    score_list=[]
    for i in range(len(user)):
        for j in range(len(user[user_index[i][1]])):
            if j%2==0:
                score_list.append(user[user_index[i][1]][j])
            else:
                location_list.append([i,int(user[user_index[i][1]][j])])
    M=len(user)
    N=max(movie)+1
    print(M,"users")
    print(N,"movies")
    print(len(score_list),"scores")
    r=3
    dataset = torch.zeros(M, N)
    score_2norm=[]
    for i in range(len(score_list)):
        dataset[location_list[i][0]][location_list[i][1]]=score_list[i]
        score_2norm.append(pow(score_list[i],2))
    user2=torch.rand(M,r)
    item2=torch.rand(N,r)
    return dataset,user2,item2,location_list,sum(score_2norm)

# Function for Artifical data to search the valid location

In [4]:
def search_location(data_tmp):
    location_list_tmp=[]
    m, n = list(data_tmp.size())
    for i in range(m):
        for j in range(n):
            if data_tmp[i][j]!=0:
                location_list_tmp.append([i,j])
    return location_list_tmp

# CGD algorithm

In [5]:
def CGD(users_tmp,items_tmp,I,loc_list,sum_s,alpha=0.01):
    start = time.process_time()
    loss_list = []
    RSME=[]
    users=deepcopy(users_tmp)
    items=deepcopy(items_tmp)
    loc_list_tmp=deepcopy(loc_list)
    for iter in range(iteration_time):
        loss=0
        for loc in loc_list_tmp:
            e=I[loc[0]][loc[1]]-torch.matmul(users[loc[0]],items[loc[1]])
            users[loc[0]]=users[loc[0]]+alpha*2*items[loc[1]]*e
            items[loc[1]]=items[loc[1]]+alpha*2*users[loc[0]]*e
        for loc_l in loc_list_tmp:
            loss += torch.norm(I[loc_l[0]][loc_l[1]] - torch.matmul(users[loc_l[0]], items[loc_l[1]]), 2)
        loss_list.append(loss)
        RSME.append(loss/sum_s)
    print(RSME[-1],"CGD Loss")
    end = time.process_time()
    print(end-start,"CGD TIME")
    return(RSME)


# CGD with Regularization Term algorithm

In [6]:
def CGD_bias(users_tmp,items_tmp,I,loc_list,sum_s,alpha=0.01,l=0.01):
    start = time.process_time()
    loss_list = []
    RSME=[]
    users=deepcopy(users_tmp)
    items=deepcopy(items_tmp)
    loc_list_tmp=deepcopy(loc_list)
    for iter in range(iteration_time):
        loss=0
        for loc in loc_list_tmp:
            e=I[loc[0]][loc[1]] - torch.matmul(users[loc[0]], items[loc[1]])
            users[loc[0]]=users[loc[0]]+alpha*2*(items[loc[1]]*e-l*users[loc[0]])
            items[loc[1]]=items[loc[1]]+alpha*2*(users[loc[0]]*e-l*items[loc[1]])
        for loc_l in loc_list_tmp:
            loss += torch.norm(I[loc_l[0]][loc_l[1]] - torch.matmul(users[loc_l[0]], items[loc_l[1]]), 2)
        loss_list.append(loss)
        RSME.append(loss/sum_s)
    print(RSME[-1],"CGD_bias Loss")
    end = time.process_time()
    print(end-start,"CGD_bias TIME")
    return(RSME)

# CGD with Armijo rules

In [7]:
def CGD_arm(users_tmp,items_tmp,I,loc_list,sum_s,s=0.03,beta=0.85,sigma = 0.375):
    start = time.process_time()
    loss_list = []
    RSME=[]
    users=deepcopy(users_tmp)
    items=deepcopy(items_tmp)
    loc_list_tmp=deepcopy(loc_list)
    for iter in range(iteration_time):
        alpha = pow(beta, iter) * s
        loss=0
        for loc in loc_list_tmp:
            e=I[loc[0]][loc[1]]-torch.matmul(users[loc[0]],items[loc[1]])
            users[loc[0]]=users[loc[0]]+sigma*alpha*2*items[loc[1]]*e
            items[loc[1]]=items[loc[1]]+sigma*alpha*2*users[loc[0]]*e
        for loc_l in loc_list_tmp:
            loss += torch.norm(I[loc_l[0]][loc_l[1]] - torch.matmul(users[loc_l[0]], items[loc_l[1]]), 2)
        loss_list.append(loss)
        RSME.append(loss/sum_s)
    print(RSME[-1],"CGD_ARM Loss")
    end = time.process_time()
    print(end-start,"CGD_ARM TIME")
    return(RSME)

# Mini-batch SGD algorithm

In [8]:
def SGD(users_tmp,items_tmp,I,loc_list,sum_s,alpha=0.01):
    start=time.process_time()
    loss_list = []
    RSME=[]
    users=deepcopy(users_tmp)
    items=deepcopy(items_tmp)
    loc_list_tmp=deepcopy(loc_list)
    n=len(loc_list)
    for iter in range(iteration_time):
        np.random.shuffle(loc_list_tmp)
        loc_list_tmp2=loc_list_tmp[0:round(0.2*n)]
        loss=0
        for loc in loc_list_tmp2:
            e=I[loc[0]][loc[1]] - torch.matmul(users[loc[0]], items[loc[1]])
            users[loc[0]]=users[loc[0]]+alpha*2*(items[loc[1]]*e)
            items[loc[1]]=items[loc[1]]+alpha*2*(users[loc[0]]*e)
        for loc_l in loc_list:
            loss+=torch.norm(I[loc_l[0]][loc_l[1]]-torch.matmul(users[loc_l[0]],items[loc_l[1]]),2)
        loss_list.append(loss)
        RSME.append(loss/sum_s)
    print(RSME[-1],"SGD RSME")
    end=time.process_time()
    print(end-start,"SGD time")
    return(RSME)

# CGD with momentum algorithm

In [9]:
def HB(users_tmp2,items_tmp2,I,loc_list,sum_s,alpha=0.01,beta=0.0001):
    start=time.process_time()
    loss_list = []
    RSME = []
    users=deepcopy(users_tmp2)
    items=deepcopy(items_tmp2)
    loc_list_tmp=deepcopy(loc_list)
    for iter in range(iteration_time):
        loss=0
        users_old=deepcopy(users)
        items_old=deepcopy(items)
        for loc in loc_list_tmp:
            e=I[loc[0]][loc[1]]-torch.matmul(users[loc[0]],items[loc[1]])
            users[loc[0]]=users[loc[0]]+alpha*2*items[loc[1]]*e+beta*(users[loc[0]]-users_old[loc[0]])
            items[loc[1]]=items[loc[1]]+alpha*2*users[loc[0]]*e+beta*(items[loc[1]]-items_old[loc[1]])
        for loc_l in loc_list:
            loss += torch.norm(I[loc_l[0]][loc_l[1]] - torch.matmul(users[loc_l[0]], items[loc_l[1]]), 2)
        loss_list.append(loss)
        RSME.append(loss / sum_s)
    print(RSME[-1], "HB RSME")
    print(time.process_time()-start,"HB TIME")
    return(RSME)

In [10]:
# Main program

In [ ]:
data,USERS,ITEMS,location_list,sum_sc=get_Netflix()
#location_list=search_location(data)
iteration_time=20
Loss_list=CGD(USERS,ITEMS,data,location_list,sum_sc)
Loss_list1=SGD(USERS,ITEMS,data,location_list,sum_sc)
Loss_list2=HB(USERS,ITEMS,data,location_list,sum_sc)
Loss_list3=CGD_bias(USERS,ITEMS,data,location_list,sum_sc)
Loss_list4=CGD_arm(USERS,ITEMS,data,location_list,sum_sc)

itr_time = range(0, iteration_time)
# itr_time2 = range(0, iteration_time)
plt.plot(itr_time, Loss_list,label='CGD with alpha=0.01')
plt.plot(itr_time, Loss_list1,label='SGD with alpha=0.01')
plt.plot(itr_time, Loss_list2,label='CGD_HB with alpha=0.01,beta=0.0001')
plt.plot(itr_time, Loss_list3,label='CGD_bias with alpha=0.01,lambda=0.01')
plt.plot(itr_time, Loss_list4,label='CGD with Armijo Rules')
plt.title('RSME v.s. Iteration for Netflix-1000000')
plt.ylabel('RSME')
plt.xlabel("Iteration time")
plt.legend()
plt.show()